<a href="https://colab.research.google.com/github/lpoggetto/fiap_liveness/blob/main/trabalho_computer_vision.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# limpando a pasta
!rm -rf /content/*

In [2]:
 # instalando pacotes utilizados

!pip install git+https://github.com/hukkelas/DSFD-Pytorch-Inference.git
!pip install face_detection
!pip install mediapipe
!pip install dlib

print('pacotes instalados')

  Cloning https://github.com/hukkelas/DSFD-Pytorch-Inference.git to /tmp/pip-req-build-fa1mzstq
  Running command git clone --filter=blob:none --quiet https://github.com/hukkelas/DSFD-Pytorch-Inference.git /tmp/pip-req-build-fa1mzstq
  Resolved https://github.com/hukkelas/DSFD-Pytorch-Inference.git to commit dde9c7dd9cdc9254c2ca345222c86a8ecfa17f5b
  Preparing metadata (setup.py) ... done
  Created wheel for face_detection: filename=face_detection-0.2.1-py3-none-any.whl size=29972 sha256=2d93c2e24fc20b50d0c467e15a1ca74262da23a0d96ec942e002ef1a67957eae
  Stored in directory: /tmp/pip-ephem-wheel-cache-l2r0ix1l/wheels/31/fc/c5/28af01da09c7625bd966f9871b081cb72e131ffb926c0de66b
Successfully built face_detection
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 55.1 MB/s eta 0:00:00
pacotes instalados


### Importando os pacotes utilizados

In [30]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import bz2
import os
import face_detection
import mediapipe as mp
from scipy.spatial import distance as dist
import dlib
from PIL import Image
import warnings
from google.colab import output
warnings.filterwarnings('ignore')

#Exibição na mesma tela do Jupyter
%matplotlib inline
sns.set_style("whitegrid", {'axes.grid' : False})

# pacotes utilizados para tirar foto/videos
from IPython.display import display, Javascript, Image, HTML
from google.colab.output import eval_js
from base64 import b64decode
import threading

dlib.DLIB_USE_CUDA = True

In [39]:
# capturando foto base para reconhecimento facial
def capturar_foto_colab():
    """Captura uma imagem no Google Colab usando a webcam."""
    display(Javascript('''
        async function captureImage() {
            const div = document.createElement('div');
            const captureButton = document.createElement('button');
            captureButton.textContent = 'Capture';
            div.appendChild(captureButton);
            const video = document.createElement('video');
            video.style.display = 'block';
            const stream = await navigator.mediaDevices.getUserMedia({video: true});
            document.body.appendChild(div);
            div.appendChild(video);
            video.srcObject = stream;
            await video.play();
            google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);
            await new Promise((resolve) => captureButton.onclick = resolve);
            const canvas = document.createElement('canvas');
            canvas.width = video.videoWidth;
            canvas.height = video.videoHeight;
            canvas.getContext('2d').drawImage(video, 0, 0);
            stream.getVideoTracks()[0].stop();
            div.remove();
            return canvas.toDataURL('image/jpeg').split(',')[1];
        }
    '''))

    with open('foto_base.jpg', 'wb') as f:
        f.write(b64decode(output.eval_js('captureImage()')))
    print("Imagem capturada e salva como 'foto_base.jpg'.")

capturar_foto_colab()

<IPython.core.display.Javascript object>

Imagem capturada e salva como 'foto_base.jpg'.


### Importando os modelos utilizados no reconhecimento facial

In [40]:
# criando diretorios para armazenar as informacoes necessarias
os.makedirs('videos', exist_ok=True) # validacao de video
os.makedirs('foto', exist_ok=True) # validacao de video

In [41]:
# URL e nome dos arquivos
landmark_url = "http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2"
landmark_file_name = "shape_predictor_68_face_landmarks.dat.bz2"
landmark_dat_file = "shape_predictor_68_face_landmarks.dat"

recognition_url = "http://dlib.net/files/dlib_face_recognition_resnet_model_v1.dat.bz2"
recognition_file_name = "dlib_face_recognition_resnet_model_v1.dat.bz2"
recognition_dat_file = "dlib_face_recognition_resnet_model_v1.dat"

# Criando diretório para salvar os modelos
os.makedirs('dlib_models', exist_ok=True)

# Apontando para o diretório
os.chdir('dlib_models')

# Função para baixar e extrair arquivos
def download_and_extract(url, compressed_file_name, extracted_file_name):
    # Baixando o arquivo se já não estiver baixado
    if not os.path.exists(compressed_file_name):
        os.system(f"wget {url}")

    # Extraindo o arquivo .dat se já não estiver extraído
    if not os.path.exists(extracted_file_name):
        with bz2.BZ2File(compressed_file_name, "rb") as f_in, open(extracted_file_name, "wb") as f_out:
            f_out.write(f_in.read())

        # Apagando o arquivo compactado
        os.remove(compressed_file_name)

# Baixando e extraindo o arquivo de landmarks
download_and_extract(landmark_url, landmark_file_name, landmark_dat_file)

# Baixando e extraindo o arquivo de reconhecimento facial
download_and_extract(recognition_url, recognition_file_name, recognition_dat_file)

# Voltando para o diretório original
os.chdir('..')

print('Modelos baixados e extraídos com sucesso!')

Modelos baixados e extraídos com sucesso!


### Tirando uma fotografia e extraindo caracteristicas para aplicar o face recognition

In [10]:
# caminho das pastas
save_folder = "/content/foto"
os.makedirs(save_folder, exist_ok=True)

# funcao para capturar foto usando javascript
def take_photo(quality=0.8):
    js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture Photo';
      capture.style.marginBottom = '10px';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      div.appendChild(video);
      document.body.appendChild(div);

      const stream = await navigator.mediaDevices.getUserMedia({video: true});
      video.srcObject = stream;
      await video.play();

      // Wait for the user to click the capture button
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getTracks().forEach(track => track.stop());
      div.remove();

      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
    display(js)
    data = eval_js('takePhoto({})'.format(quality))
    binary = b64decode(data.split(',')[1])
    return binary

# capturar a foto
photo_binary = take_photo()

# nome do arquivo
file_name = input("Enter the file name (with .jpg extension): ")

# garantindo que esta salvo como jpg
if not file_name.lower().endswith('.jpg'):
    file_name += '.jpg'

file_path = os.path.join(save_folder, file_name)
with open(file_path, 'wb') as f:
    f.write(photo_binary)

print(f"Photo saved to {file_path}")

<IPython.core.display.Javascript object>

Enter the file name (with .jpg extension): lucas_poggetto
Photo saved to /content/foto/lucas_poggetto.jpg


'/content/foto/lucas_poggetto.jpg'

In [ ]:
def validar_reconhecimento_colab(foto_base_path="file_path"):
    """Valida o reconhecimento facial comparando a imagem capturada no Colab com a imagem base."""
    try:
        detector = dlib.get_frontal_face_detector()
        reconhecedor = dlib.face_recognition_model_v1("dlib_face_recognition_resnet_model_v1.dat")
        predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
    except RuntimeError as e:
        print("Erro ao carregar os modelos do dlib. Certifique-se de que a versão CPU está instalada corretamente ou desative o uso de CUDA.")
        print(e)
        return

    # Carregar imagem base e extrair descritor
    imagem_base = cv2.imread(foto_base_path)  # Corrigir indentação aqui
    faces_base = detectar_face(imagem_base, detector)

    if len(faces_base) == 0:
        raise Exception("Nenhuma face detectada na imagem base.")

    shape_base = predictor(imagem_base, faces_base[0])
    descritor_base = reconhecedor.compute_face_descriptor(imagem_base, shape_base)

    print("Capturando imagem para validação.")
    capturar_foto_colab()

    imagem_live = cv2.imread('foto_base.jpg')
    faces_live = detectar_face(imagem_live, detector)

    if len(faces_live) == 0:
        raise Exception("Nenhuma face detectada na imagem ao vivo.")

    shape_live = predictor(imagem_live, faces_live[0])
    descritor_live = reconhecedor.compute_face_descriptor(imagem_live, shape_live)

    # Comparar os descritores
    distancia = np.linalg.norm(np.array(descritor_base) - np.array(descritor_live))
    print(f"Distância entre os descritores: {distancia:.2f}")

    if distancia < 0.6:
        print("Faces correspondem! Acesso permitido.")
    else:
        print("Faces não correspondem! Acesso negado.")

In [ ]:
# JavaScript code to capture video
html_code = """
<video id="video" width="640" height="480" autoplay></video>
<button id="startButton">Start Recording</button>
<button id="stopButton" disabled>Stop Recording</button>
<video id="playback" width="640" height="480" controls></video>
<a id="downloadLink" download="recorded-video.webm"></a>

<script>
    const video = document.querySelector('#video');
    const playback = document.querySelector('#playback');
    const downloadLink = document.querySelector('#downloadLink');
    const startButton = document.querySelector('#startButton');
    const stopButton = document.querySelector('#stopButton');

    navigator.mediaDevices.getUserMedia({ video: true }).then((stream) => {
        video.srcObject = stream;
        let mediaRecorder;
        const chunks = [];

        startButton.onclick = () => {
            mediaRecorder = new MediaRecorder(stream);
            mediaRecorder.start();
            startButton.disabled = true;
            stopButton.disabled = false;

            mediaRecorder.ondataavailable = (event) => chunks.push(event.data);
            mediaRecorder.onstop = () => {
                const blob = new Blob(chunks, { type: 'video/webm' });
                playback.src = URL.createObjectURL(blob);
                downloadLink.href = playback.src;
                downloadLink.textContent = 'Download Video';
                startButton.disabled = false;
                stopButton.disabled = true;
            };
        };

        stopButton.onclick = () => mediaRecorder.stop();
    });
</script>
"""

# Display the HTML interface
display(HTML(html_code))

### Montando detector de faces

Utilizando o algoritmo DSFDDetector para detectar uma face

In [ ]:
def calculo_distancia_olhos(eye):
    # Calcula a distância euclidiana entre os pontos dos olhos
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])
    C = dist.euclidean(eye[0], eye[3])
    # Calcula a razão de aspecto do olho
    ear = (A + B) / (2.0 * C)
    return ear

def detector_faces_video(video_path, output_path, threshhold_ear=0.3):
    # Criando o detector de faces
    detector = face_detection.build_detector(
        "DSFDDetector",
        confidence_threshold=0.5,
        nms_iou_threshold=0.3
    )
    # utilizando algoritmo de 68 pontos faciais
    preditor_marcos_faciais = dlib.shape_predictor(
        '/content/dlib_models/shape_predictor_68_face_landmarks.dat'
        )

    # abrindo o arquivo de video
    video = cv2.VideoCapture(video_path)

    # propriedades do video
    width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fourcc = cv2.VideoWriter_fourcc(*'VP80')  # WebM codec
    out = cv2.VideoWriter(output_path, fourcc, 15, (width, height))

    while True:
        ret, frame = video.read()
        if not ret:
            break

        # convertendo para RGB
        rgb_frame = frame[:, :, ::-1]

        # detectando  face
        detections = detector.detect(rgb_frame)

        # desenhando o retangulo na face
        for det in detections:
            x1, y1, x2, y2, score = det
            x1, y1, x2, y2 = map(int, [x1, y1, x2, y2])

            # Draw a red rectangle by default
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 2)

            # utilizando dlib para detectar a face
            dlib_rect = dlib.rectangle(x1,y1,x2,y2)

            # marcos faciais
            landmarks = preditor_marcos_faciais(rgb_frame, dlib_rect)

            # coordenadas dos olhos
            left_eye = [(landmarks.part(i).x, landmarks.part(i).y) for i in range(36, 42)]
            right_eye = [(landmarks.part(i).x, landmarks.part(i).y) for i in range(42, 48)]

            # calculando a distancia para ambos os olhos
            olho_esq = calculo_distancia_olhos(left_eye)
            olho_dir = calculo_distancia_olhos(right_eye)
            ear = (olho_esq + olho_dir) / 2

            # validacao de olhos piscando
            blink_detected = ear < threshhold_ear

            # Check if blink is detected
            if ear < threshhold_ear:
                # Update the rectangle to green if liveness is proven
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame, 'Liveness OK', (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        # retorno do video
        out.write(frame)

    # Release resources
    video.release()
    out.release()
    print(f"Input video FPS: {fps}")
    print(f"Processed video saved to: {output_path}")

# Example usage
input_video_path = '/content/videos/recorded-video.webm'
output_video_path = '/content/videos/recorded-video_processado.webm'

# chamada da funcao criada anteriormente
detector_faces_video(input_video_path, output_video_path, threshhold_ear = 0.3)

Input video FPS: 62.5
Processed video saved to: /content/videos/recorded-video_processado.webm
